In [2]:
import pandas as pd
import datetime as dt

# 1. Load Data
df = pd.read_excel('Online Retail.xlsx')

# 2. Clean Data
df = df.dropna(subset=['CustomerID'])
df = df[~df['InvoiceNo'].astype(str).str.startswith('C')]
df = df[df['Quantity'] > 0]
df['TotalAmount'] = df['Quantity'] * df['UnitPrice']

# 3. Calculate RFM
latest_date = df['InvoiceDate'].max() + dt.timedelta(days=1)
rfm = df.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (latest_date - x.max()).days,
    'InvoiceNo': 'nunique',
    'TotalAmount': 'sum'
}).reset_index()
rfm.columns = ['CustomerID', 'Recency', 'Frequency', 'Monetary']

# 4. Display the Head (This will now look like a nice table!)
display(rfm.head())

,CustomerID,Recency,Frequency,Monetary
0,12346.0,326,1,77183.60
1,12347.0,2,7,4310.00
2,12348.0,75,4,1797.24
3,12349.0,19,1,1757.55
4,12350.0,310,1,334.40


In [4]:
# 1. Drop rows with no CustomerID (we can't segment ghosts)
df = df.dropna(subset=['CustomerID'])

# 2. Remove Cancelled orders (InvoiceNo starts with 'C')
df = df[~df['InvoiceNo'].astype(str).str.startswith('C')]

# 3. Remove negative quantities/prices
df = df[(df['Quantity'] > 0) & (df['UnitPrice'] > 0)]

# 4. Create a 'TotalAmount' column (Money Spent)
df['TotalAmount'] = df['Quantity'] * df['UnitPrice']

print(f"Cleaned Data Rows: {df.shape[0]}")

Cleaned Data Rows: 397884


In [6]:
# Set the reference date to 1 day after the last transaction
latest_date = df['InvoiceDate'].max() + dt.timedelta(days=1)

# Group by CustomerID to calculate RFM
rfm = df.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (latest_date - x.max()).days, # Recency
    'InvoiceNo': 'nunique',                                # Frequency
    'TotalAmount': 'sum'                                   # Monetary
}).reset_index()

# Rename columns
rfm.columns = ['CustomerID', 'Recency', 'Frequency', 'Monetary']

print(rfm.head())



   CustomerID  Recency  Frequency  Monetary
0     12346.0      326          1  77183.60
1     12347.0        2          7   4310.00
2     12348.0       75          4   1797.24
3     12349.0       19          1   1757.55
4     12350.0      310          1    334.40


In [8]:
# 1. Create Scores (1-5 Scale)
rfm['R_Score'] = pd.qcut(rfm['Recency'], 5, labels=[5, 4, 3, 2, 1]) # 5 is Best Recency (Low days)
rfm['F_Score'] = pd.qcut(rfm['Frequency'].rank(method='first'), 5, labels=[1, 2, 3, 4, 5]) # 5 is Best Frequency

# 2. Define the Segment Map (Regex Logic)
seg_map = {
    r'[1-2][1-2]': 'Hibernating',
    r'[1-2][3-4]': 'At Risk',
    r'[1-2]5': 'Cannot Lose',
    r'3[1-2]': 'About To Sleep',
    r'33': 'Need Attention',
    r'[3-4][4-5]': 'Loyal Customers',
    r'41': 'Promising',
    r'51': 'New Customers',
    r'[4-5][2-3]': 'Potential Loyalists',
    r'5[4-5]': 'Champions'
}

# 3. Apply the Map
rfm['Segment'] = (rfm['R_Score'].astype(str) + rfm['F_Score'].astype(str)).replace(seg_map, regex=True)

# Show the results
print(rfm['Segment'].value_counts())

Segment
Hibernating            1065
Loyal Customers         827
Champions               633
At Risk                 580
Potential Loyalists     492
About To Sleep          351
Need Attention          186
Promising                99
Cannot Lose              63
New Customers            42
Name: count, dtype: int64


In [9]:
rfm.to_csv('RFM_Final_Data.csv', index=False)
print("File 'RFM_Final_Data.csv' has been saved to your folder!")

File 'RFM_Final_Data.csv' has been saved to your folder!
